#### CREATE EXTERNAL LOCATION

In [0]:
CREATE EXTERNAL LOCATION IF NOT EXISTS pei_retail_externallocation
URL 'abfss://retail@peiretailstorageaccount.dfs.core.windows.net/'
WITH (STORAGE CREDENTIAL pei_storage_credential)
COMMENT 'External location for Pei Assessment Project'

#### CREATE CATALOG

In [0]:
CREATE CATALOG IF NOT EXISTS pei
MANAGED LOCATION 'abfss://retail@peiretailstorageaccount.dfs.core.windows.net/'
COMMENT 'This is the catalog for the PEI Assessment Project' 

#### CREATE SCHEMAS

#####1. Bronze Schema

In [0]:
use catalog pei;

CREATE SCHEMA IF NOT EXISTS bronze
MANAGED LOCATION 'abfss://retail@peiretailstorageaccount.dfs.core.windows.net/delta/bronze'
COMMENT 'This is the bronze schema for the PEI Assessment Project';

#####2. Silver Schema

In [0]:
use catalog pei;

CREATE SCHEMA IF NOT EXISTS silver
MANAGED LOCATION 'abfss://retail@peiretailstorageaccount.dfs.core.windows.net/delta/silver'
COMMENT 'This is the silver schema for the PEI Assessment Project';

#####3. Gold Schema

In [0]:
use catalog pei;

CREATE SCHEMA IF NOT EXISTS gold
MANAGED LOCATION 'abfss://retail@peiretailstorageaccount.dfs.core.windows.net/delta/gold'
COMMENT 'This is the gold schema for the PEI Assessment Project';

#### CREATE VOLUMES

#####1. Landing Volume

In [0]:
Use catalog pei;
use schema default;

CREATE EXTERNAL VOLUME IF NOT EXISTS landing_volume
LOCATION 'abfss://retail@peiretailstorageaccount.dfs.core.windows.net/landing'
COMMENT 'landing volume';

#####2. Checkpoint Volume

In [0]:
Use catalog pei;
use schema default;

CREATE EXTERNAL VOLUME IF NOT EXISTS checkpoint_volume
LOCATION 'abfss://retail@peiretailstorageaccount.dfs.core.windows.net/checkpoints'
COMMENT 'checkpoints volume';

#####1. Order Volumes

In [0]:
Use catalog pei;
use schema default;

CREATE EXTERNAL VOLUME IF NOT EXISTS raw_orders_schema_checkpoint
LOCATION 'abfss://pei@dbwnyctaxi.dfs.core.windows.net/sales/checkpoints/raw_orders_schema_checkpoint'
COMMENT 'Raw Orders Schema Checkpoint';

CREATE EXTERNAL VOLUME IF NOT EXISTS raw_orders_file_checkpoint
LOCATION 'abfss://pei@dbwnyctaxi.dfs.core.windows.net/sales/checkpoints/raw_orders_file_checkpoint'
COMMENT 'Raw Orders File Checkpoint';

#####2. Product Volumes

In [0]:
Use catalog pei;
use schema default;

CREATE EXTERNAL VOLUME IF NOT EXISTS raw_products_schema_checkpoint
LOCATION 'abfss://pei@dbwnyctaxi.dfs.core.windows.net/sales/checkpoints/raw_products_schema_checkpoint'
COMMENT 'Raw Products Schema Checkpoint';

CREATE EXTERNAL VOLUME IF NOT EXISTS raw_products_file_checkpoint
LOCATION 'abfss://pei@dbwnyctaxi.dfs.core.windows.net/sales/checkpoints/raw_products_file_checkpoint'
COMMENT 'Raw Products File Checkpoint';

#####3. Raw Data Volume

In [0]:
Use catalog pei;
use schema default;

CREATE EXTERNAL VOLUME IF NOT EXISTS raw_data_volume
LOCATION 'abfss://pei@dbwnyctaxi.dfs.core.windows.net/sales/data'
COMMENT 'Raw Data Volume';

#### CREATE DEFAULT TABLES

#####1. Create WaterMark Table

In [0]:
Use catalog pei;
use schema default;

DROP TABLE IF EXISTS batch_watermark;

CREATE TABLE batch_watermark (
    table_name STRING,
    last_ingestion_ts TIMESTAMP,
    last_processed_version INTEGER
)

#### CREATE BRONZE TABLES

#####1. Create raw_orders Table

In [0]:
Use catalog pei;
USe SCHEMA bronze;

DROP TABLE IF EXISTS raw_orders;

CREATE TABLE IF NOT EXISTS raw_orders
(
  `Row ID` STRING,
  `Order ID` STRING,
  `Order Date` STRING,
  `Ship Date` STRING,
  `Ship Mode` STRING,
  `Customer ID` string,
  `Product ID` STRING,
  `Quantity` STRING,
  `Price` STRING, 
  `Discount` STRING, 
  `Profit` STRING,
   file_path STRING,
   ingestion_timestamp TIMESTAMP,
   `_rescued_data` STRING
)
USING DELTA
TBLPROPERTIES 
(
  'delta.columnMapping.mode' = 'name',
  delta.autoOptimize.optimizeWrite = true,
  delta.autoOptimize.autoCompact = true
)
CLUSTER BY AUTO

#####2. Create raw_products Table

In [0]:
Use catalog pei;
USe SCHEMA bronze;

DROP TABLE IF EXISTS raw_products;

CREATE TABLE IF NOT EXISTS raw_products
(
  `Product ID` STRING,
  Category STRING,
  `Sub-Category` STRING,
  `Product Name` STRING,
  `State` STRING,
  `Price per product` string,
   file_path STRING,
   ingestion_timestamp TIMESTAMP,
   `_rescued_data` STRING
)
USING DELTA
TBLPROPERTIES 
(
  'delta.columnMapping.mode' = 'name',
  delta.autoOptimize.optimizeWrite = true,
  delta.autoOptimize.autoCompact = true
)
CLUSTER BY AUTO

#####3. Create raw_customers Table

In [0]:
Use catalog pei;
USe SCHEMA bronze;

DROP TABLE IF EXISTS raw_customers;

CREATE TABLE IF NOT EXISTS raw_customers
(
  `Customer ID` STRING,
  `Customer Name` STRING,
  email STRING,
  phone STRING,
  address STRING,
  Segment string,
  Country STRING, 
  City STRING, 
  State STRING, 
  `Postal Code` STRING,
  `Region` STRING,
  file_path STRING,
  ingestion_timestamp TIMESTAMP
)
USING DELTA
TBLPROPERTIES 
(
  'delta.columnMapping.mode' = 'name',
  delta.autoOptimize.optimizeWrite = true,
  delta.autoOptimize.autoCompact = true
)
CLUSTER BY AUTO

#### CREATE SILVER TABLES

#####1. Create Enriched Customer SCD1

In [0]:
use catalog pei;
use schema silver;

DROP TABLE IF EXISTS customers_enriched;

CREATE TABLE customers_enriched
(
  customer_id string, 
  customer_name string,
  first_name string, 
  last_name string, 
  email string, 
  phone string, 
  address string, 
  segment string, 
  country string, 
  city string, 
  state string, 
  postal_code string, 
  `region` string,

  --metadata columns 
  file_path STRING,              
  ingestion_timestamp TIMESTAMP, 
  processing_timestamp TIMESTAMP
)

#####2. Create Enriched Product SCD1

In [0]:
USE CATALOG pei;
USE SCHEMA silver;

DROP TABLE IF EXISTS products_enriched;

CREATE TABLE products_enriched (
    product_id STRING,
    category STRING,
    sub_category STRING,
    product_name STRING,
    state STRING,
    price_per_product DECIMAL(10,2),

    --metadata columns 
    file_path STRING,
    ingestion_timestamp TIMESTAMP,
    processing_timestamp TIMESTAMP 
)

#####3. Create Quaratine Orders Table

In [0]:
USE CATALOG pei;
USE SCHEMA silver;

DROP TABLE IF EXISTS orders_quarantine;

CREATE TABLE orders_quarantine 
(
  row_id int, 
  quarantine_reason STRING,
  severity_level STRING, 

  --metadata columns 
  file_path STRING,
  ingestion_timestamp TIMESTAMP,
  processing_timestamp TIMESTAMP 
)
TBLPROPERTIES(
  delta.autoOptimize.optimizeWrite = true,
  delta.autoOptimize.autoCompact = true
)

#####4. Create Order Enriched Table

In [0]:
USE CATALOG pei;
USE SCHEMA silver;

DROP TABLE IF EXISTS orders_enriched;

CREATE TABLE orders_enriched 
(
    order_id STRING,
    order_date TIMESTAMP,
    year_month STRING GENERATED ALWAYS AS (date_format(order_date, 'yyyy-MM')),
    category STRING,
    sub_category STRING,
    customer_name STRING,
    country STRING,
    profit Decimal(10, 2)
)
PARTITIONED BY (year_month)
TBLPROPERTIES
(
    delta.enableChangeDataFeed = true,
    delta.autoOptimize.optimizeWrite = true,
    delta.autoOptimize.autoCompact = true
);

CREATE BLOOMFILTER INDEX ON orders_enriched FOR COLUMNS (order_id);

#### CREATE GOLD TABLES

#####1. Create Sales Aggregate Table

In [0]:
USE CATALOG pei;
USE SCHEMA gold;

DROP TABLE IF EXISTS agg_sales_performance;

CREATE TABLE agg_sales_performance 
(
    order_year STRING,
    category STRING,
    sub_category STRING,
    customer_name STRING,
    total_profit Decimal(10, 2)
)
PARTITIONED BY (order_year)
TBLPROPERTIES(
    delta.autoOptimize.optimizeWrite = true,
    delta.autoOptimize.autoCompact = true
)

--OPTIMIZE pei.gold.agg_sales_performance 
--ZORDER BY (customer_name, category);